# Problem(a)

In [ ]:
# from matplotlib import pyplot
import os

#define a function to count the document number 
def Count(dirname):
  count=0
  for item in os.listdir(dirname):
    count+=1
  return count

counter=[]

#plot the histogram for 20 topics
for dir in os.listdir('20news-bydate-train'):
  dir_name='20news-bydate-train/'+dir
  count1=Count(dir_name)
  counter.append(count1)

i=0;

for dir in os.listdir('20news-bydate-test'):
  #print counter[i],'\n'
  dir_name='20news-bydate-test/'+dir
  count2=Count(dir_name)
  counter[i]+=count2
  #print counter[i],'\n'
  i+=1
  
#count the document number for the two groups
new_counter=[0,0]
for dir in os.listdir('20news-bydate-train'):
  if(dir=='comp.graphics' or dir=='comp.os.ms-windows.misc' or dir=='comp.sys.ibm.pc.hardware' or dir=='comp.sys.mac.hardware'):
    dir_name='20news-bydate-train/'+dir
    count1=Count(dir_name)
    new_counter[0]+=count1
    #print (new_counter[0],'\n')
  if(dir=='rec.autos' or dir=='rec.motorcycles' or dir=='rec.sport.baseball' or dir=='rec.sport.hockey'):
    dir_name='20news-bydate-train/'+dir
    count1=Count(dir_name)
    new_counter[1]+=count1
    #print (new_counter[1],'\n')
    
for dir in os.listdir('20news-bydate-test'):
  if(dir=='comp.graphics' or dir=='comp.os.ms-windows.misc' or dir=='comp.sys.ibm.pc.hardware' or dir=='comp.sys.mac.hardware'):
    dir_name='20news-bydate-test/'+dir
    count2=Count(dir_name)
    new_counter[0]+=count2
    #print (new_counter[0],'\n')
  if(dir=='rec.autos' or dir=='rec.motorcycles' or dir=='rec.sport.baseball' or dir=='rec.sport.hockey'):
    dir_name='20news-bydate-test/'+dir
    count2=Count(dir_name)
    new_counter[1]+=count2
    #print (new_counter[1],'\n')

print ('Group1 is: ',new_counter[0],', Group2 is: ',new_counter[1])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
n = 20
values = (counter[0],counter[1],counter[2],counter[3],counter[4],counter[5],counter[6],counter[7],counter[8],counter[9],counter[10],counter[11],counter[12],counter[13],counter[14],counter[15],counter[16],counter[17],counter[18],counter[19])
fig, ax = plt.subplots()
index = np.arange(n)
bar_width = 0.35
opacity = 1.0
bars = plt.bar(index, values, bar_width,alpha = opacity, color = 'b')
plt.xlabel('Topics')
plt.xlabel('\nDocuments')
plt.title('Number of documents per topic')
plt.xticks(index + bar_width, ('atheism', 'graphics', 'windows.\nmisc', 'pc.\nhardware', 'mac.\nhardware', 'windows.x', 'forsale', 'autos', 'motorcycles', 'sport.\nbaseball', 'sport.\nhockey', 'crypt', 'electronics', 'med', 'space', 'christian', 'guns', 'mideast', 'politics.\nmisc', 'religion.\nmisc'))
plt.ylim(0,1000)

plt.legend()
plt.tight_layout()
plt.show()

# Problem(b)

In [1]:
from sklearn.feature_extraction import text
import re
import nltk 
import string
#nltk.download()

stop_words = text.ENGLISH_STOP_WORDS

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenizer_fun(text):
    new_text = re.sub(r'[^A-Za-z]', " ", text)
    tokens =[word for sent in nltk.sent_tokenize(new_text) for word in nltk.word_tokenize(sent)]
    new_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]{2,}', token):
            new_tokens.append(token)     
    stem = [stemmer.stem(t) for t in new_tokens]
    return stem

from sklearn.datasets import fetch_20newsgroups
all_data = fetch_20newsgroups(subset='all', shuffle=True, random_state=42, remove=('headers','footers','quotes'))

from sklearn.feature_extraction.text import TfidfVectorizer
TFxIDF = TfidfVectorizer(analyzer='word',tokenizer=tokenizer_fun, stop_words=stop_words,token_pattern='[a-zA-Z]{2,}',)
TFxIDF_data = TFxIDF.fit_transform(all_data.data)
count = TFxIDF_data.shape

print (count)

(18846, 72399)


# Problem(c)

In [2]:
my_list=['comp.sys.ibm.pc.hardware','comp.sys.mac.hardware','misc.forsale','soc.religion.christian']

for category in my_list:
    categories = [category]
    sub_data = fetch_20newsgroups(subset='all', categories=categories, shuffle=True, random_state=42, remove=('headers','footers','quotes'))
    TFxIDF_sub = TfidfVectorizer(analyzer='word',tokenizer=tokenizer_fun, max_features=10, stop_words=stop_words,token_pattern='[a-zA-Z]{2,}')
    sub_count = TFxIDF_sub.fit_transform(sub_data.data)
    #print (sub_count)
    print (TFxIDF_sub.vocabulary_.keys())
    

[u'drive', u'problem', u'disk', u'work', u'card', u'mb', u'ani', u'scsi', u'use', u'control']
[u'ani', u'drive', u'know', u'like', u'problem', u'work', u'use', u'mac', u'appl', u'monitor']
[u'new', u'offer', u'drive', u'ship', u'sale', u'sell', u'includ', u'price', u'pleas', u'use']
[u'peopl', u'say', u'believ', u'church', u'jesus', u'think', u'god', u'christian', u'sin', u'know']


# Problem(d)

In [3]:
from sklearn.decomposition import TruncatedSVD
X = TruncatedSVD(n_components=50, algorithm='arpack')
LSI = X.fit_transform(TFxIDF_data)
#print (LSI)

# Problem(e)

#### Define functions to get data or LSI for train/test set

In [42]:
categories = ['comp.graphics','comp.os.ms-windows.misc','comp.sys.ibm.pc.hardware','comp.sys.mac.hardware','rec.autos','rec.motorcycles','rec.sport.baseball','rec.sport.hockey']

def data_fun(train_or_test):
    if(train_or_test == 'train'):
        data = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42, remove=('headers','footers','quotes'))
    if(train_or_test == 'test'):
        data = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42, remove=('headers','footers','quotes'))
    return data  

def LSI_fun(train_or_test):
    if(train_or_test == 'train'):
        data = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42, remove=('headers','footers','quotes'))
        TFxIDF_sub = TfidfVectorizer(analyzer='word',tokenizer=tokenizer_fun, stop_words=stop_words,token_pattern='[a-zA-Z]{2,}',)
        TFxIDF_sub_data = TFxIDF_sub.fit_transform(data.data)
        Y = TruncatedSVD(n_components=50, algorithm='arpack')
        sub_LSI = Y.fit_transform(TFxIDF_sub_data)
    if(train_or_test == 'test'):
        data = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42, remove=('headers','footers','quotes'))
        TFxIDF_sub = TfidfVectorizer(analyzer='word',tokenizer=tokenizer_fun, stop_words=stop_words,token_pattern='[a-zA-Z]{2,}',)
        TFxIDF_sub_data = TFxIDF_sub.fit_transform(data.data)
        Y = TruncatedSVD(n_components=50, algorithm='arpack')
        sub_LSI = Y.fit_transform(TFxIDF_sub_data)
    return sub_LSI 

#new_LSI = LSI_fun('train')
#print (new_LSI.shape)

#new_LSI = LSI_fun('test')
#print (new_LSI.shape)

#### SVM classifier

In [44]:
from sklearn import svm
lin_clf = svm.LinearSVC()
# print(all_data.target)
train_LSI = LSI_fun('train')
train_set = data_fun('train')

train_target_group = [ int(x / 4) for x in train_set.target]
# print(train_target_group)
#print(train_set.target[0:20])
# lin_clf.fit(train_LSI, train_set.target)
lin_clf.fit(train_LSI, train_target_group)
#lin_clf.fit(train_LSI, train_set.target)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [45]:
test_LSI = LSI_fun('test')
print(test_LSI.shape)
svm_predicted = lin_clf.predict(test_LSI)
print (svm_predicted[0:10])

(3150, 50)
[1 0 0 0 0 0 0 1 0 1]


#### SVM-ROC Curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
test_set = data_fun('test')
test_target_group = [ int(x / 4) for x in test_set.target]
y_score_test = lin_clf.decision_function(test_LSI)

#print(test_target_group[0:10])
#print(y_score_test.shape)
#print(y_score_test[0:10])

fpr, tpr, thresholds = roc_curve(test_target_group, y_score_test)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, lw=1, label='ROC fold (area = %0.2f)' % roc_auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC_SVM Curve')
plt.legend(loc="lower right")
plt.show()

#### SVM-Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_target_group, svm_predicted)

#### SVM-Accuracy, Precision, Recall

In [ ]:
from sklearn.metrics import accuracy_score
svm_accuracy = accuracy_score(test_target_group, svm_predicted)
print (svm_accuracy)

In [ ]:
from sklearn.metrics import precision_score
svm_precision_score = precision_score(test_target_group, svm_predicted)
print (svm_precision_score)

In [ ]:
from sklearn.metrics import recall_score
svm_recall_score = recall_score(test_target_group, svm_predicted)
print (svm_recall_score)

#### SVM-Recall and Precision

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
precision = dict()
recall = dict()
average_precision = dict()
precision, recall, _ = precision_recall_curve(test_target_group,y_score_test)
average_precision = average_precision_score(test_target_group, y_score_test)
#print (precision[0:10])
#print (recall[0:10])

# Problem(f)

### Find the best parameter lambda

In [ ]:
from sklearn.cross_validation import KFold
kf = KFold(len(train_target_group), n_folds=5,shuffle=False,random_state=None)
len(kf)
print(kf)

In [ ]:
from sklearn import svm
#initialize a matrix with 7 rows(lamda) and 5 columns(5 validation)
scores = [[0]*7 for i in range(5)]
i = 0
j = 0

for train_index, test_index in kf:
    X_train, X_test = train_LSI[train_index], train_LSI[test_index]
    for k in [-3, -2, -1, 0, 1, 2, 3]:
        X_train_target_group = [ int(x / 4) for x in train_set.target[train_index]]
        X_test_target_group = [ int(x / 4) for x in train_set.target[test_index]]
        
        soft_svm_clf = svm.LinearSVC(C=10**k)
        soft_svm_clf.fit(X_train, X_train_target_group)
        scores[i][j]= soft_svm_clf.score(X_test, X_test_target_group)        
        j=j+1
    i=i+1
    j=0


In [ ]:
ave_score = list(map(lambda x: (x[0]+x[1]+x[2]+x[3]+x[4])/5, zip(scores[0], scores[1], scores[2], scores[3], scores[4])))
max_score = max(ave_score)
index=ave_score.index(max_score)
r = [-3, -2, -1, 0, 1, 2, 3]
print ('The best lambda is',10**r[index])

### Find the best parameter lambda  (another method)

In [ ]:
from sklearn import svm
from sklearn import cross_validation
import numpy as np
penalty = [-3, -2, -1, 0, 1, 2, 3]
for k in penalty:
    soft_svm_clf = svm.LinearSVC(C=10**k)
    scores = cross_validation. cross_val_score(soft_svm_clf, train_LSI, train_target_group, cv=5)
    print np.mean(scores)
max_score = max(scores)
index=scores.index(max_score)
penalty = [-3, -2, -1, 0, 1, 2, 3]
print ('The best lambda is',10**penalty[index])

### Soft_SVM classifier

In [ ]:
soft_clf = svm.LinearSVC(C=10**1)
soft_clf.fit(train_LSI, train_target_group)
soft_svm_predicted = soft_clf.predict(test_LSI)

### Soft_SVM-ROC Curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
test_set = data_fun('test')
test_target_group = [ int(x / 4) for x in test_set.target]
y_score_test_soft_svm = soft_clf.decision_function(test_LSI)

#print(test_target_group[0:10])
#print(y_score_test.shape)
#print(y_score_test[0:10])

fpr, tpr, thresholds = roc_curve(test_target_group, y_score_test_soft_svm)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, lw=1, label='ROC fold (area = %0.2f)' % roc_auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC_SVM_soft Curve')
plt.legend(loc="lower right")
plt.show()

### Soft_SVM Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_target_group, soft_svm_predicted)

### Soft_clf-Accuracy, Precision, Recall

In [ ]:
from sklearn.metrics import accuracy_score
soft_clf_accuracy = accuracy_score(test_target_group, soft_svm_predicted)
print (soft_clf_accuracy)

In [ ]:
from sklearn.metrics import precision_score
soft_clf_precision_score = precision_score(test_target_group, soft_svm_predicted)
print (soft_clf_precision_score)

In [ ]:
from sklearn.metrics import recall_score
soft_clf_recall_score = recall_score(test_target_group, soft_svm_predicted)
print (soft_clf_recall_score)

# Problem(g)

#### Gaussian-NB classifier

In [43]:
from sklearn.naive_bayes import GaussianNB
nb_clf = GaussianNB()
nb_clf.fit(train_LSI, train_target_group)
nb_predicted = nb_clf.predict(test_LSI)
print (nb_predicted.shape)
print (nb_predicted[0:10])

NameError: name 'train_LSI' is not defined

#### Gaussian-NB-ROC Curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
test_set = data_fun('test')
test_target_group = [ int(x / 4) for x in test_set.target]
y_score_test_nb = nb_clf.predict_proba(test_LSI)[:,1]

#print(y_score_test_nb.shape)
#print(y_score_test_nb[0:10])

fpr, tpr, thresholds = roc_curve(test_target_group, y_score_test_nb)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, lw=1, label='ROC fold (area = %0.2f)' % roc_auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC_NB Curve')
plt.legend(loc="lower right")
plt.show()

#### Gaussian-NB-Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_target_group, nb_predicted)

#### Gaussian-NB-Accuracy, Precision, Recall

In [ ]:
from sklearn.metrics import accuracy_score
nb_accuracy = accuracy_score(test_target_group, nb_predicted)
print (nb_accuracy)

In [ ]:
from sklearn.metrics import precision_score
nb_precision_score = precision_score(test_target_group, nb_predicted)
print (nb_precision_score)

In [ ]:
from sklearn.metrics import recall_score
nb_recall_score = recall_score(test_target_group, nb_predicted)
print (nb_recall_score)

#### Gaussian-NB-Recall and Precision

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
y_score_test_nb = nb_clf.predict_proba(test_LSI)[:,1]
precision = dict()
recall = dict()
average_precision = dict()
precision, recall, _ = precision_recall_curve(test_target_group,y_score_test_nb)
average_precision = average_precision_score(test_target_group, y_score_test_nb)
#print (precision[0:10])
#print (recall[0:10])

#### Bernoulli-NB classifier

In [46]:
from sklearn.naive_bayes import GaussianNB
nb_clf = BernoulliNB()
nb_clf.fit(train_LSI, train_target_group)
nb_predicted = nb_clf.predict(test_LSI)
print (nb_predicted.shape)
print (nb_predicted[0:10])

(3150,)
[0 0 0 0 0 1 0 1 1 0]


#### Bernoulli-NB-ROC Curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
test_set = data_fun('test')
test_target_group = [ int(x / 4) for x in test_set.target]
y_score_test_nb = nb_clf.predict_proba(test_LSI)[:,1]

#print(y_score_test_nb.shape)
#print(y_score_test_nb[0:10])

fpr, tpr, thresholds = roc_curve(test_target_group, y_score_test_nb)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, lw=1, label='ROC fold (area = %0.2f)' % roc_auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC_NB Curve')
plt.legend(loc="lower right")
plt.show()

#### Bernoulli-NB-Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_target_group, nb_predicted)

#### Bernoulli-NB-Accuracy, Precision, Recall

In [ ]:
from sklearn.metrics import accuracy_score
nb_accuracy = accuracy_score(test_target_group, nb_predicted)
print (nb_accuracy)

In [ ]:
from sklearn.metrics import precision_score
nb_precision_score = precision_score(test_target_group, nb_predicted)
print (nb_precision_score)

In [ ]:
from sklearn.metrics import recall_score
nb_recall_score = recall_score(test_target_group, nb_predicted)
print (nb_recall_score)

#### Bernoulli-NB-Recall and Precision

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
y_score_test_nb = nb_clf.predict_proba(test_LSI)[:,1]
precision = dict()
recall = dict()
average_precision = dict()
precision, recall, _ = precision_recall_curve(test_target_group,y_score_test_nb)
average_precision = average_precision_score(test_target_group, y_score_test_nb)
#print (precision[0:10])
#print (recall[0:10])

# Problem(h)

#### LR classifier

In [ ]:
from sklearn import linear_model, datasets
logreg = linear_model.LogisticRegression(C=1e5)
logreg.fit(train_LSI, train_target_group)
lr_predicted = logreg.predict(test_LSI)
print (lr_predicted.shape)
print (lr_predicted[0:10])

#### LR-ROC Curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
test_set = data_fun('test')
test_target_group = [ int(x / 4) for x in test_set.target]
y_score_test_lr = logreg.decision_function(test_LSI)
#y_score_test_lr = logreg.predict_proba(test_LSI)[:,1]

#print(y_score_test_lr.shape)
#print(y_score_test_lr[0:10])

fpr, tpr, thresholds = roc_curve(test_target_group, y_score_test_lr)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, lw=1, label='ROC fold (area = %0.2f)' % roc_auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC_LR Curve')
plt.legend(loc="lower right")
plt.show()

#### LR-Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_target_group, lr_predicted)

#### LR-Accuracy, Precision, Recall

In [ ]:
from sklearn.metrics import accuracy_score
lr_accuracy = accuracy_score(test_target_group, lr_predicted)
print (lr_accuracy)

In [ ]:
from sklearn.metrics import precision_score
lr_precision_score = precision_score(test_target_group, lr_predicted)
print (lr_precision_score)

In [ ]:
from sklearn.metrics import recall_score
lr_recall_score = recall_score(test_target_group, lr_predicted)
print (lr_recall_score)

#### LR-Recall and Precision

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
y_score_test_lr = logreg.decision_function(test_LSI)
precision = dict()
recall = dict()
average_precision = dict()
precision, recall, _ = precision_recall_curve(test_target_group,y_score_test_lr)
average_precision = average_precision_score(test_target_group, y_score_test_lr)
#print (precision[0:10])
#print (recall[0:10])

# Part (i)

#### Define functions to get data or LSI for train/test set

In [6]:
categories_i = ['comp.sys.ibm.pc.hardware','comp.sys.mac.hardware','misc.forsale','soc.religion.christian']

def data_fun_i(train_or_test):
    if(train_or_test == 'train'):
        data = fetch_20newsgroups(subset='train', categories=categories_i, shuffle=True, random_state=42, remove=('headers','footers','quotes'))
    if(train_or_test == 'test'):
        data = fetch_20newsgroups(subset='test', categories=categories_i, shuffle=True, random_state=42, remove=('headers','footers','quotes'))
    return data

def LSI_fun_i(train_or_test):
    if(train_or_test == 'train'):
        data = fetch_20newsgroups(subset='train', categories=categories_i, shuffle=True, random_state=42, remove=('headers','footers','quotes'))
        TFxIDF_sub = TfidfVectorizer(analyzer='word',tokenizer=tokenizer_fun, stop_words=stop_words,token_pattern='[a-zA-Z]{2,}',)
        TFxIDF_sub_data = TFxIDF_sub.fit_transform(data.data)
        Y = TruncatedSVD(n_components=50, algorithm='arpack')
        sub_LSI = Y.fit_transform(TFxIDF_sub_data)
    if(train_or_test == 'test'):
        data = fetch_20newsgroups(subset='test', categories=categories_i, shuffle=True, random_state=42, remove=('headers','footers','quotes'))
        TFxIDF_sub = TfidfVectorizer(analyzer='word',tokenizer=tokenizer_fun, stop_words=stop_words,token_pattern='[a-zA-Z]{2,}',)
        TFxIDF_sub_data = TFxIDF_sub.fit_transform(data.data)
        Y = TruncatedSVD(n_components=50, algorithm='arpack')
        sub_LSI = Y.fit_transform(TFxIDF_sub_data)
    return sub_LSI

In [7]:
from sklearn.multiclass import OneVsRestClassifier 
from sklearn.multiclass import OneVsOneClassifier 
from sklearn.svm import LinearSVC
train_LSI_i = LSI_fun_i('train')
train_set_i = data_fun_i('train')
test_LSI_i = LSI_fun_i('test')
test_set_i = data_fun_i('test')
#train_target_group_i = [ int(x / 10) for x in train_set_i.target]
#test_target_group_i = [int(x / 10) for x in test_set_i.target]

## one vs rest classifier

In [ ]:
ovr_classifier_i = OneVsRestClassifier (LinearSVC())
ovr_classifier_i.fit(train_LSI_i, train_set_i.target)
ovr_predict_i = ovr_classifier_i.predict(test_LSI_i)
#print (ovr_predict_i[0:4])
print (ovr_predict_i)

#### OVR-Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_set_i.target, ovr_predict_i)

#### OVR-Accuracy, Precision, Recall

In [ ]:
from sklearn.metrics import accuracy_score
ovr_accuracy = accuracy_score(test_set_i.target, ovr_predict_i)
print (ovr_accuracy)

In [ ]:
from sklearn.metrics import precision_score
ovr_precision_score = precision_score(test_set_i.target, ovr_predict_i,average='weighted')
print (ovr_precision_score)

In [ ]:
from sklearn.metrics import recall_score
ovr_recall_score = recall_score(test_set_i.target, ovr_predict_i,average='weighted')
print (ovr_recall_score)

## one vs one classifier

In [ ]:
ovo_classifier_i = OneVsOneClassifier (LinearSVC())
ovo_classifier_i.fit(train_LSI_i, train_set_i.target)
ovo_predict_i = ovo_classifier_i.predict(test_LSI_i)
print (ovo_predict_i.shape)
print (ovo_predict_i)

#### OVO-Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_set_i.target, ovo_predict_i)

#### OVO-Accuracy, Precision, Recall

In [ ]:
from sklearn.metrics import accuracy_score
ovo_accuracy = accuracy_score(test_set_i.target, ovo_predict_i)
print (ovo_accuracy)

In [ ]:
from sklearn.metrics import precision_score
ovo_precision_score = precision_score(test_set_i.target, ovo_predict_i,average='weighted')
print (ovo_precision_score)

In [ ]:
from sklearn.metrics import recall_score
ovo_recall_score = recall_score(test_set_i.target, ovo_predict_i,average='weighted')
print (ovo_recall_score)

## Gaussian NB Multiclass Classifier

In [9]:
from sklearn.naive_bayes import GaussianNB
nb_classifier_i = GaussianNB()
nb_classifier_i.fit(train_LSI_i, train_set_i.target)
nb_predict_i = nb_classifier_i.predict(test_LSI_i)
print (nb_predict_i.shape)
print (nb_predict_i[0:4])

(1565,)
[2 0 3 2]


#### Gaussian-NB-Multi Confusion Matrix

In [15]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_set_i.target, nb_predict_i)

array([[137, 161,  72,  22],
       [109, 127, 105,  44],
       [ 79,  45, 204,  62],
       [ 97,  18,  56, 227]])

#### Gaussian-NB-Multi-Accuracy, Precision, Recall

In [16]:
from sklearn.metrics import accuracy_score
nb_accuracy = accuracy_score(test_set_i.target, nb_predict_i)
print (nb_accuracy)

0.444089456869


In [17]:
from sklearn.metrics import precision_score
nb_precision_score = precision_score(test_set_i.target, nb_predict_i,average='weighted')
print (nb_precision_score)

0.449276631965


In [18]:
from sklearn.metrics import recall_score
nb_recall_score = recall_score(test_set_i.target, nb_predict_i,average='weighted')
print (nb_recall_score)

0.444089456869


## Bernoulli NB Multiclass Classifier

In [19]:

from sklearn.naive_bayes import BernoulliNB
nb_classifier_i = BernoulliNB()
nb_classifier_i.fit(train_LSI_i, train_set_i.target)
nb_predict_i = nb_classifier_i.predict(test_LSI_i)
print (nb_predict_i.shape)
print (nb_predict_i[0:4])

(1565,)
[2 0 3 2]


#### Bernoulli-NB-Multi Confusion Matrix

In [36]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_set_i.target, nb_predict_i)

array([[162, 169,  54,   7],
       [189,  89,  80,  27],
       [ 57,  23, 296,  14],
       [ 10,  27,  48, 313]])

#### Bernoulli-NB-Multi-Accuracy, Precision, Recall

In [37]:
from sklearn.metrics import accuracy_score
nb_accuracy = accuracy_score(test_set_i.target, nb_predict_i)
print (nb_accuracy)

0.549520766773


In [38]:
from sklearn.metrics import precision_score
nb_precision_score = precision_score(test_set_i.target, nb_predict_i,average='weighted')
print (nb_precision_score)

0.542977669991


In [39]:
from sklearn.metrics import recall_score
nb_recall_score = recall_score(test_set_i.target, nb_predict_i,average='weighted')
print (nb_recall_score)

0.549520766773


## Multinomial NB Multiclass Classifier

In [40]:
from sklearn.naive_bayes import MultinomialNB
nb_classifier_i = MultinomialNB()
nb_classifier_i.fit(train_LSI_i, train_set_i.target)
nb_predict_i = nb_classifier_i.predict(test_LSI_i)
print (nb_predict_i.shape)
print (nb_predict_i[0:4])

ValueError: Input X must be non-negative

#### Multinomial-NB-Multi Confusion Matrix

In [31]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_set_i.target, nb_predict_i)

array([[162, 169,  54,   7],
       [189,  89,  80,  27],
       [ 57,  23, 296,  14],
       [ 10,  27,  48, 313]])

#### Bernoulli-NB-Multi-Accuracy, Precision, Recall

In [32]:
from sklearn.metrics import accuracy_score
nb_accuracy = accuracy_score(test_set_i.target, nb_predict_i)
print (nb_accuracy)

0.549520766773


In [33]:
from sklearn.metrics import precision_score
nb_precision_score = precision_score(test_set_i.target, nb_predict_i,average='weighted')
print (nb_precision_score)

0.542977669991


In [34]:
from sklearn.metrics import recall_score
nb_recall_score = recall_score(test_set_i.target, nb_predict_i,average='weighted')
print (nb_recall_score)

0.549520766773
